### https://mlflow.org/docs/latest/deep-learning/tensorflow/quickstart/quickstart_tensorflow.html

## install dependencies


In [ ]:
#!pip install -q mlflow

In [ ]:
# import the packages
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras

### Load the MNIST dataset

In [ ]:
# Load the mnist dataset.
train_ds, test_ds = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
)

##### Let’s preprocess our data with the following steps: - Scale each pixel’s value to [0, 1).
##### Batch the dataset. 
##### Use prefetch to speed up the training.

In [ ]:
def preprocess_fn(data):
    image = tf.cast(data["image"], tf.float32) / 255
    label = data["label"]
    return (image, label)


train_ds = train_ds.map(preprocess_fn).batch(128).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess_fn).batch(128).prefetch(tf.data.AUTOTUNE)

### Define the model

In [ ]:
input_shape = (28, 28, 1)
num_classes = 10

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

model.summary()

### Setup MLFlow autologging

In [ ]:
import mlflow

# connect the experiment to the tracking server
mlflow.set_tracking_uri("http://localhost:5000")

# set the experiment name
mlflow.set_experiment("mlflow-tf-keras-mnist")

# enable autologging
mlflow.tensorflow.autolog()
mlflow.autolog()

### Train the model for a few epochs

In [ ]:
model.fit(x=train_ds, epochs=10)

### Let’s evaluate the training result.

In [ ]:
score = model.evaluate(test_ds)

print(f"Test loss: {score[0]:.4f}")
print(f"Test accuracy: {score[1]: .2f}")

### Log with MLflow Callback

In [ ]:
from mlflow.tensorflow import MLflowCallback

# Turn off autologging.
mlflow.tensorflow.autolog(disable=True)

with mlflow.start_run(run_name='bisleri') as run:
    model.fit(
        x=train_ds,
        epochs=10,
        callbacks=[MLflowCallback(run)],
    )

### Customize the MLflow Callback

In [ ]:
# If you want to add extra logging logic, you can customize the MLflow callback. 
# You can either subclass from keras.callbacks.Callback and write everything from scratch or 
# subclass from mlflow.tensorflow.MLflowCallback to add you custom logging logic.

import math
from mlflow import log_metrics

# Create our own callback by subclassing `MLflowCallback`.
class MLflowCustomCallback(MLflowCallback):
    def on_epoch_end(self, epoch, logs=None):
        if not self.log_every_epoch:
            return
        loss = logs["loss"]
        logs["log_loss"] = math.log(loss)
        del logs["loss"]
        log_metrics(logs, epoch)
        #self.metrics_logger.record_metrics(logs, epoch)

In [ ]:
# Train the model with the new callback.

with mlflow.start_run() as run:
    run_id = run.info.run_id
    model.fit(
        x=train_ds,
        epochs=5,
        callbacks=[MLflowCustomCallback(run)],
    )